In [163]:
#!/usr/bin/python3
import pandas as pd
import json

import influxdb_client
from influxdb import DataFrameClient
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS

from dateutil import parser
import requests
from string import Template


In [164]:
# Use only following fields from CSV. 
Fields = ['TimeStamp','School','Class','Students','Absent']
# Define tag fields
datatags = ['School','Class']
# Define fixed tags
fixtags = {"Country": "India", "State": "Haryana", "City": "Kurukshetra"}

# Read data from CSV without index and parse 'TimeStamp' as date.
df = pd.read_csv("sample.csv", sep=';', index_col=False, parse_dates=['TimeStamp'], usecols=Fields)

# Set 'TimeStamp' field as index of dataframe
df.set_index('TimeStamp', inplace = True)

print(df.head())

                              School Class  Students  Absent
TimeStamp                                                   
2020-05-09 17:45:00+00:00      USSMS   UKG        30       3
2020-05-09 17:45:00+00:00      USSMS    10        50       2
2020-05-09 17:45:00+00:00      USSMS     8        40       0
2020-05-09 18:45:00+00:00  St Thomas   Nur        30       4
2020-05-09 18:45:00+00:00  St Thomas     6        55       4


In [165]:
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS

bucket = "DSI_test"
token ="yxk8_Or5qHZJxrhJE3SkAnTQSViQCsmrUoR0xPZd_0scy1T8FTuL1cKSTDKh1ft8Bqs3Zbt7Rwkys-FzajIVFQ=="
org="DSI"
client = InfluxDBClient(url="http://localhost:8086", token=token, org=org)

In [166]:
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

In [167]:

from influxdb_client.client.write_api import ASYNCHRONOUS

records = pd.DataFrame(
    data=[
        ["0", 25.3, 1657729063],
    	  ["0", 25.4, 1657729078],
    	  ["0", 25.2, 1657729093],
    ],
    columns=["core", "temp", "timestamp"],
)
records["timestamp"] = pd.to_datetime(records["timestamp"], unit="s")
records.set_index("timestamp", inplace=True)
#records.drop(colums=["core"], inplace=True)

with client.write_api(write_options=ASYNCHRONOUS) as writer:
    writer.write(
        bucket=bucket,
        record=records,
        data_frame_measurement_name="cpu3",
        data_frame_tag_columns=["core"]
        )

In [168]:
with client.write_api(write_options=ASYNCHRONOUS) as writer:
    writer.write(
        bucket=bucket,
        record=df,
        data_frame_measurement_name="student",
        data_frame_tag_columns=datatags,
        data_frame_field_columns=["Students", "Absent"]
        )


In [169]:
symbols = ['IBM']

url = Template('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=$symbol&interval=5min&apikey=5LZM13OZQMZFPU92')

for symbol in symbols:
    r = requests.get(url.substitute(symbol=symbol))
    data = r.json()

    #print(data)

#make json to dataframe

df_IBM = pd.DataFrame(data['Time Series (5min)']).T


In [170]:
df_IBM.columns = ['open', 'high', 'low', 'close', 'volume']

#change col types to float

df_IBM = df_IBM.astype(float)


In [171]:
with client.write_api(write_options=ASYNCHRONOUS) as writer:
    writer.write(
        bucket=bucket,
        record=df_IBM,
        data_frame_measurement_name="IBM_2",
        data_frame_tag_columns=["symbol"],
        data_frame_field_columns=["open", "high", "low", "close", "volume"]
        )

In [172]:
df_IBM

,open,high,low,close,volume
2023-01-19 19:50:00,140.7200,140.7200,140.7200,140.7200,151.0
2023-01-19 18:50:00,140.6200,140.6200,140.6200,140.6200,610.0
2023-01-19 18:45:00,140.6500,140.6500,140.6000,140.6400,744.0
2023-01-19 17:45:00,140.7999,140.7999,140.7999,140.7999,254.0
2023-01-19 17:35:00,140.8000,140.8000,140.6200,140.6200,524.0
...,...,...,...,...,...
2023-01-19 07:25:00,140.0500,140.0500,140.0200,140.0200,941.0
2023-01-19 07:10:00,140.1400,140.1400,140.1400,140.1400,501.0
2023-01-19 06:40:00,140.1400,140.1600,140.1400,140.1600,1364.0
2023-01-18 18:40:00,140.2000,140.2000,140.2000,140.2000,100.0
